In [1]:
import os
import sys
print(sys.executable)

/Users/andrei/anaconda3/envs/cil/bin/python


In [2]:
from gensim.models.word2vec import Word2Vec

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
TRAIN = os.path.join('..', 'train')
POS_TWEET_FILE = os.path.join(TRAIN, 'train_pos_full.txt')
NEG_TWEET_FILE = os.path.join(TRAIN, 'train_pos_full.txt')

In [4]:
def read_tweets(fname):
    """Read the tweets in the given file.
    
    Returns a 2d array where every row is a tweet, split into words.
    """
    with open(fname, 'r') as f:
        return [l.split() for l in f.readlines()]

In [5]:
pos_tweets = read_tweets(POS_TWEET_FILE)

In [6]:
neg_tweets = read_tweets(NEG_TWEET_FILE)

In [12]:
sentences = pos_tweets + neg_tweets
print(len(sentences))

2500000


In [81]:
WORKERS = 8
# Note: Moises's team uses size=200 as of June 13.
# See: https://groups.google.com/forum/#!msg/gensim/ggCHGncd5rU/Z_pQDD69AAAJ
# for some parameter hints.
model = Word2Vec(sentences, size=300, window=10, min_count=5, workers=WORKERS)# , alpha=0.05, cbow_mean=1)

In [82]:
# Should be queen
model.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.4797830581665039),
 ('alpha', 0.3895503878593445),
 ('madonna', 0.37537628412246704),
 ('protects', 0.3463192582130432),
 ('hawk', 0.33808666467666626),
 ('lawrence', 0.3364061117172241),
 ('lion', 0.33565545082092285),
 ('jennifer', 0.32868629693984985),
 ('child', 0.32832834124565125),
 ('kenny', 0.3242822587490082)]

In [83]:
# Should be germany
model.most_similar(positive=['france', 'berlin'], negative=['paris'])

[('croatia', 0.6175637245178223),
 ('norway', 0.5980886816978455),
 ('switzerland', 0.5956143140792847),
 ('germany', 0.5937309265136719),
 ('sweden', 0.5898008346557617),
 ('denmark', 0.584825873374939),
 ('belgium', 0.5835149884223938),
 ('russia', 0.5813409090042114),
 ('montreal', 0.579700231552124),
 ('poland', 0.5749874114990234)]

In [84]:
model.doesnt_match("breakfast cereal dinner lunch".split())

'cereal'

In [85]:
model.estimate_memory()

{'syn0': 83107200, 'syn1neg': 83107200, 'total': 200842400, 'vocab': 34628000}

In [76]:
print(model.similarity('woman', 'man'))
print(model.similarity('woman', 'coffee'))
print(model.similarity('woman', 'penis'))
print(model.similarity('woman', 'football'))

0.515749259372
0.103649427512
0.23129938755
-0.0222698924543


In [77]:
model.similarity('car','man')

0.18013204849313583

In [78]:
model.similarity('car','truck')

0.64671567731553359

In [79]:
# Download this for testing: https://github.com/arfon/word2vec/blob/master/questions-words.txt
# Highly recommended!

question_file = "questions-words.txt"
accuracy_results = model.accuracy(question_file)

In [80]:
summary = accuracy_results[-1]
assert summary['section'] == 'total'
incorrect = summary['incorrect']
correct = summary['correct']

incorrect_n = len(incorrect)
correct_n = len(correct)

acc = correct_n / incorrect_n
print("{0:5.3f} accuracy; Analogies: {1} correct, {2} incorrect".format(
    acc, correct_n, incorrect_n))

0.342 accuracy; Analogies: 2356 correct, 6881 incorrect


### Accuracies (full Twitter data)
 * Vanilla (size=225, window=5, min_count=5): 0.319 accuracy; Analogies: 2233 correct, 7004 incorrect
 * (size=300, window=5,  min_count=5): 0.337 accuracy; Analogies: 2329 correct, 6908 incorrect
 * (size=500, window=5,  min_count=5): 0.330 accuracy; Analogies: 2292 correct, 6945 incorrect
 * (size=300, window=10, min_count=5): 0.346 accuracy; Analogies: 2374 correct, 6863 incorrect
 * (size=300, window=15, min_count=5): 0.342 accuracy; Analogies: 2356 correct, 6881 incorrect



In [86]:
# TODO(andrei): Save the model!
fname = "./word2vec-local-gensim.bin"
model.save(fname)